In [7]:
import os
import numpy as np
import importlib
import torch
from torch_geometric.data import Data
from pygod.detector import DOMINANT
from pygod.detector import CoLA
#from pygod.detector import dominant_ckgconv
from pygod.metric import eval_roc_auc, eval_f1, eval_average_precision, eval_precision_at_k, eval_recall_at_k
from pygod.utils import load_data
from utils.optuna_tuner import tune_model_with_optuna, sanitize_params
from utils.param_spaces import ParamSpace
from utils.eval_repeat import eval_repeat

# Datasets

In [2]:
def load_and_prepare_dataset(path):
    data = torch.load(path)
    data.x = data.x.float()
    data.edge_index = data.edge_index.long()
    data.y = data.y.long()
    return data

def label_to_binary(y):
    return (y != 0).long()

def label_identity(y):
    return y.long()

In [3]:
paths = {
    "inj_cora": "/data1/shengen/STATS403/project/data/real/inj_cora.pt",
    "books": "/data1/shengen/STATS403/project/data/real/books.pt",
    "gen_500": "/data1/shengen/STATS403/project/data/synthetic/gen_500.pt",
}


datasets = {}
for name, path in paths.items():
    datasets[name] = load_and_prepare_dataset(path)

for name, data in datasets.items():
    print(f"{name}:")
    print(" - x:", data.x.shape)
    print(" - edge_index:", data.edge_index.shape)
    print(" - y unique:", torch.unique(data.y).tolist())
    print()

inj_cora:
 - x: torch.Size([2708, 1433])
 - edge_index: torch.Size([2, 11060])
 - y unique: [0, 1, 2, 3]

books:
 - x: torch.Size([1418, 21])
 - edge_index: torch.Size([2, 3695])
 - y unique: [0, 1]

gen_500:
 - x: torch.Size([500, 64])
 - edge_index: torch.Size([2, 2662])
 - y unique: [0, 1, 2]



In [4]:
datasets_to_run = {
    "inj_cora": {
        "data": datasets["inj_cora"],
        "label_fn": label_to_binary
    },
    "books": {
        "data": datasets["books"],
        "label_fn": label_identity
    },
    "gen_500": {
        "data": datasets["gen_500"],
        "label_fn": label_to_binary
    }
}

os.makedirs("results", exist_ok=True)

# Train DOMINANT

In [ ]:
for name, config in datasets_to_run.items():
    print(f"\nRunning DOMINANT on {name}...\n")

    param_space_fn = ParamSpace.get("dominant")

    study = tune_model_with_optuna(
        model_class=DOMINANT,
        dataset=config["data"],
        label_transform_fn=config["label_fn"],
        param_space_fn=param_space_fn,
        n_trials=30,
        train_epochs=30,
        metric="auc",
        device=0,
        return_all_metrics=True,
        dataset_name=name  
    )

    best_params = sanitize_params(study.best_params)

    print(f"Best params for DOMINANT on {name}: {best_params}")

    binary_label = config["label_fn"](config["data"].y)

    summary, df = eval_repeat(
        model_class=DOMINANT,
        dataset=config["data"],
        binary_label=binary_label,
        params=best_params,
        n_runs=10,
        epochs=200,
        save_path=f"results/dominant_{name}.csv"
    )

    print(f"Summary for DOMINANT on {name}:")
    for k, v in summary.items():
        print(f"  {k}: {v}")


[I 2025-04-21 13:37:44,652] A new study created in memory with name: no-name-38d121cf-388a-46c6-abaa-91c30b03fc53


 Running DOMINANT on inj_cora...



  0%|          | 0/30 [00:00<?, ?it/s]/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
Best trial: 0. Best value: 0.756172:   3%|▎         | 1/30 [00:00<00:13,  2.17it/s]

[Trial 0] AUC: 0.7562 | Params: {'hid_dim': 32, 'num_layers': 2, 'dropout': 0.07248785746688224, 'lr': 0.00010487692440254823, 'weight': 0.1206095649804066}
[I 2025-04-21 13:37:45,113] Trial 0 finished with value: 0.7561721084982801 and parameters: {'hid_dim': 32, 'num_layers': 2, 'dropout': 0.07248785746688224, 'lr': 0.00010487692440254823, 'weight': 0.1206095649804066}. Best is trial 0 with value: 0.7561721084982801.


/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
Best trial: 0. Best value: 0.756172:   7%|▋         | 2/30 [00:01<00:14,  1.94it/s]

...

[Run 9/10]


/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


[Run 10/10]


/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


Saved results to results/dominant_gen_500.csv
Summary for gen_500:
  AUC-ROC (mean±std): 0.7955 ± 0.0018
  AP (mean±std): 0.5208 ± 0.0017
  F1 (mean±std): 0.3143 ± 0.0000


# Train CoLA

In [ ]:
for name, config in datasets_to_run.items():
    print(f"\nRunning CoLA on {name}...\n")

    param_space_fn = ParamSpace.get("cola")

    study = tune_model_with_optuna(
        model_class=CoLA,
        dataset=config["data"],
        label_transform_fn=config["label_fn"],
        param_space_fn=param_space_fn,
        n_trials=30,
        train_epochs=30,
        metric="auc",
        device=0,
        return_all_metrics=True,
        dataset_name=name  
    )

    best_params = sanitize_params(study.best_params)

    print(f"Best params for CoLA on {name}: {best_params}")

    binary_label = config["label_fn"](config["data"].y)

    summary, df = eval_repeat(
        model_class=CoLA,
        dataset=config["data"],
        binary_label=binary_label,
        params=best_params,
        n_runs=10,
        epochs=200,
        save_path=f"results/cola_{name}.csv"
    )

    print(f"Summary for CoLA on {name}:")
    for k, v in summary.items():
        print(f"  {k}: {v}")


[I 2025-04-21 15:13:24,487] A new study created in memory with name: no-name-0a827bbb-be93-47e9-8410-747b696192a7



Running CoLA on inj_cora...



  0%|          | 0/30 [00:00<?, ?it/s]/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
Best trial: 0. Best value: 0.540274:   3%|▎         | 1/30 [00:00<00:27,  1.04it/s]/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


[I 2025-04-21 15:13:25,450] Trial 0 finished with value: 0.5402737833417921 and parameters: {'hid_dim': 32, 'num_layers': 4, 'dropout': 0.30487802570938094, 'lr': 0.0011799085367456797, 'weight_decay': 0.0015003098327317844, 'activation': 'elu'}. Best is trial 0 with value: 0.5402737833417921.


Best trial: 0. Best value: 0.540274:   7%|▋         | 2/30 [00:01<00:15,  1.80it/s]/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


...

[Run 9/10]


/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


[Run 10/10]


/data1/shengen/conda_envs/ckgconv_env/lib/python3.11/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


Saved results to results/cola_gen_500.csv
Summary for CoLA on gen_500:
  AUC-ROC (mean±std): 0.4899 ± 0.0419
  AP (mean±std): 0.0521 ± 0.0151
  F1 (mean±std): 0.0657 ± 0.0287


# Train CKGConv_DOMINANT

In [ ]:
for name, config in datasets_to_run.items():
    print(f"\nRunning CKGConv-DOMINANT on {name}...\n")

    param_space_fn = ParamSpace.get("ckgconv_dominant")

    study = tune_model_with_optuna(
        model_class=CKGConvDOMINANT,
        dataset=config["data"],
        label_transform_fn=config["label_fn"],
        param_space_fn=param_space_fn,
        n_trials=30,
        train_epochs=30,
        metric="auc",
        device=0,
        return_all_metrics=True,
        dataset_name=name  
    )

    best_params = sanitize_params(study.best_params)

    print(f"Best params for CKGConv-DOMINANT on {name}: {best_params}")

    binary_label = config["label_fn"](config["data"].y)

    summary, df = eval_repeat(
        model_class=CKGConvDOMINANT,
        dataset=config["data"],
        binary_label=binary_label,
        params=best_params,
        n_runs=10,
        epochs=200,
        save_path=f"results/ckgconv_dominant_{name}.csv"
    )

    print(f"Summary for CKGConv-DOMINANT on {name}:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

In [ ]:
# CKGConv_dominant on books

In [ ]:
# CKGConv_dominant on synthetic

In [ ]:
# visualization